In [13]:
from __future__ import division
import os
import numpy as np
import pandas as pd
from skimage.io import imread, imshow, imsave, show
from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', -1)

## Load data

In [14]:
expected_tags = {}
all_tags = set(["agriculture","artisinal_mine","bare_ground","blooming","blow_down","clear","cloudy","conventional_mine","cultivation","habitation","haze","partly_cloudy","primary","road","selective_logging","slash_burn","water"])
with open("./rawInput/train-augmented.csv", "r") as train_labels:
    train_labels.readline()
    for line in train_labels:
        img, tags = line.rstrip("\n").split(",")
        expected_tags[img] = tags.split(" ")
print len(expected_tags)

323832


In [19]:
predicted_tags = {}
with open("./predict/train-predict.csv", "r") as predict_labels:
    predict_labels.readline()
    for line in predict_labels:
        img, tags = line.rstrip("\n").split(",")
        predicted_tags[img] = tags.split(" ")
print len(predicted_tags)

323832


In [20]:
predicted_proba = {}
with open("./predict/train-predict-raw.csv", "r") as predict_proba:
    tags = predict_proba.readline().rstrip("\n").split(",")[1].split(" ")
    for line in predict_proba:
        img, proba = line.rstrip("\n").split(",")
        proba = proba.split(" ")
        predicted_proba[img] = { t: float(v) for t, v in zip(tags, proba) }
print len(predicted_tags)

323832


In [21]:
training_set = set([])
validation_set = set([])
with open("./train/training-files.csv", "r") as training_files:
    original_training_set = set(training_files.readline().strip().split(","))
    for f in os.listdir("./rawInput/train-jpg"):
        filename = f.split(".")[0]
        s = training_set if filename in original_training_set else validation_set
        for i in range(8):
            s.add("{name}--{i}".format(name=filename, i=i))

print "original images:", len(os.listdir("./rawInput/train-jpg"))
print "total images:", len(os.listdir("./rawInput/train-jpg-augmented"))
print "training set:", len(training_set)
print "validation set:", len(validation_set)

original images: 40479
total images: 323832
training set: 291448
validation set: 32384


## Read predictions

### Per label score

In [26]:
# First value is number of correctly predicted
# Second is number of not detected (false negative)
# Third is number of inaccurately predicted (false positive)
tags_score = {t: { "tp":0, "fn": 0, "fp": 0} for t in all_tags}

def scores(tp, fp, fn):
    p = tp / (tp + fp)
    r = tp / (tp + fn)
    b = 2
    if p == 0 or r == 0:
        f2 = 0
    else:
        b = 2
        f2 = (1 + b**2) * p * r / (b**2*p + r)
    return p, r, f2
    
for img in validation_set:
    true_tags = expected_tags[img]
    pred_tags = predicted_tags[img]
    for t in true_tags:
        if t in pred_tags:
            tags_score[t]["tp"] += 1
        else:
            tags_score[t]["fn"] += 1
    for t in pred_tags:
        if t not in true_tags:
            tags_score[t]["fp"] += 1

data = []
for tag in all_tags:
    s = tags_score[tag]
    p, r, f2 = scores(s["tp"], s["fp"], s["fn"])
    data.append([tag, s["tp"], s["fp"], s["fn"], p, r, f2])
data.sort(key = lambda l:l[-1])
data = np.array(data)
columns = ["tp", "fp", "fn", "validation precision", "validation recall", "f2"]
df = pd.DataFrame(data=data[:, 1:], index=data[:, 0], columns=columns)
df

,tp,fp,fn,validation precision,validation recall,f2
slash_burn,17,57,159,0.22972972973,0.0965909090909,0.109254498715
blooming,36,182,124,0.165137614679,0.225,0.20979020979
bare_ground,193,249,575,0.43665158371,0.251302083333,0.274615822425
selective_logging,91,125,181,0.421296296296,0.334558823529,0.348926380368
cultivation,1847,1287,2017,0.589342693044,0.478002070393,0.496772458311
blow_down,46,42,34,0.522727272727,0.575,0.563725490196
conventional_mine,74,35,46,0.678899082569,0.616666666667,0.62818336163
haze,1430,716,826,0.666356011184,0.633865248227,0.640107430618
habitation,1854,787,906,0.7020068156,0.671739130435,0.677582048096
water,4187,1080,1621,0.794949686729,0.720902203857,0.734587178497


## F2 score

In [27]:
LABELS = ['clear', 'cloudy', 'haze', 'partly_cloudy', 'agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 'conventional_mine', 'cultivation', 'habitation', 'primary', 'road', 'selective_logging', 'slash_burn', 'water']

In [28]:
import numpy as np
from sklearn.metrics import fbeta_score
def F2Score(predicted, actual):
    # see https://www.kaggle.com/c/planet-understanding-the-amazon-from-space#evaluation
    predicted = set(predicted)
    actual = set(actual)
    tp = len(predicted & actual)
    tn = len(LABELS) - len(predicted | actual)
    fp = len(predicted) - tp
    fn = (len(LABELS) - len(predicted)) - tn
    return scores(tp, fp, fn)[2]

def scoreSet(data_set):
    scores = []
    f2, n = 0, 0
    for img in data_set:
        real_tags = expected_tags[img]
        pred_tags = predicted_tags[img]
        s = F2Score(real_tags, pred_tags)
        f2 += s
        scores.append({ "img": img, "f2": s })
        n += 1
    return f2 / n, scores

In [29]:
training_score, training_scores = scoreSet(training_set)
validation_score, validation_scores = scoreSet(validation_set)
training_scores.sort(key = lambda l: l["f2"])
validation_scores.sort(key = lambda l: l["f2"])

print "F2 score for training set", training_score
print "F2 score validation set", validation_score

F2 score for training set 0.994026038266
F2 score validation set 0.894884424695


## Prediction per image

In [48]:
short_labels = {"agriculture": "ag","artisinal_mine": "a_mi","bare_ground": "b_gr","blooming": "bloom","blow_down": "blow","clear": "clea","cloudy": "clou","conventional_mine": "c_mi","cultivation": "cult","habitation": "hab","haze": "haze","partly_cloudy": "p_clo","primary": "prim","road": "road","selective_logging": "s_log","slash_burn": "sl_b","water": "wat"}

# return an HTML display for one prediction entry, making it green, red or grey
def getLabelPred(tag, img):
    truth = tag in expected_tags[img]
    pred = tag in predicted_tags[img]
    prob = predicted_proba[img][tag]
    isGood = truth == pred
    if truth == pred:
        cl = "green" if truth else "grey"
    else:
        cl = "red true" if truth else "red false"
    return '<span title="{tag}" class="{cl}">{val}</span>'.format(tag=tag, val=round(prob, 2), cl=cl)
        
# Return a dataframe the predictions
def getPrediction(data):
    img = data["img"]
    f2 = data["f2"]
    image = '<img src="./rawInput/train-jpg-augmented/{img}.jpg" class="input-image"/>'.format(img=img)
    
    return [
        img, 
        image, 
        f2
    ] + [getLabelPred(t, img) for t in all_tags]

table = []
originals = set([])
for data in validation_scores[:1000]:
    img = data["img"]
    original = img.split("--")[0]
    if original not in originals:
        table.append(getPrediction(data))
        originals.add(original)

table = np.array(table)

columns = ["image", "f2"] + [short_labels[t] for t in all_tags]
df = pd.DataFrame(data=table[:, 1:], index=table[:, 0], columns=columns)
style = "<style>.input-image{width: 50px; max-width: 250px !important;} .input-image:hover{width: 250px;} .green{color: green;} .grey{color: #e0e0e0;} .red{color: black;} .true{border-bottom: 3px solid green;} .false{border-bottom: 3px solid red;}</style>"
HTML(style + df.to_html(escape=False))

,image,f2,sl_b,s_log,cult,clea,hab,c_mi,clou,prim,wat,haze,a_mi,p_clo,bloom,b_gr,blow,ag,road
train_36726--5,,0,0.0,0.0,0.0,0.01,0.0,0.0,0.83,0.09,0.0,0.16,0.0,0.02,0.0,0.0,0.0,0.46,0.0
train_16397--5,,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
train_19637--6,,0,0.0,0.0,0.0,0.07,0.0,0.0,0.47,0.16,0.51,0.02,0.0,0.05,0.0,0.0,0.0,0.32,0.0
train_30702--3,,0,0.0,0.0,0.02,0.0,0.64,0.0,0.0,0.99,0.98,0.0,0.0,1.0,0.0,0.0,0.0,0.91,0.1
train_36562--2,,0,0.0,0.0,0.0,0.0,0.0,0.0,0.94,0.06,0.0,0.01,0.0,0.14,0.0,0.0,0.0,0.0,0.0
train_15364--0,,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.54,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.01,0.0
train_9741--3,,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
train_18154--7,,0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.39,0.63,0.2,0.0,0.06,0.0,0.0,0.0,0.0,0.07
train_6522--1,,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0
train_29424--4,,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
